In [1]:
# Test whether the two task in the given asymmetry experiment have significantly different line fitting

In [2]:
from urllib.request import urlopen
import numpy as np
import sys
from rt_plot import gen_RT_data
import os
from tqdm.notebook import tqdm
import pandas as pd

import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.graphics.api import interaction_plot, abline_plot
from statsmodels.stats.anova import anova_lm

In [3]:
border = "".join(100*["-"]) + "\n"

rt_fxn_model = {'slope': 252.359, 'intercept': 376.271}
task_ids = [*range(6)]

In [4]:
testData = []

methods = np.array(os.listdir("./eccNET/out/")+["pixelMatch", "chance", "gbvs"])
dirs = ['./eccNET/out/']*(len(methods)-3)+["./"]*3
for n in range(len(methods)):
    method = methods[n]
    all_fxn_data, all_fxn_seq = gen_RT_data(method, dirs[n], rt_fxn_model, task_ids)
    
    header = ["Model"]
    modelTestData = [method]
    for t in range(len(all_fxn_seq)):
        data = []

        for i in range(len(all_fxn_seq[t])):
            fxn = all_fxn_data[all_fxn_seq[t][i]]
            exp_name = fxn.name

            rt = fxn.num_fxn*rt_fxn_model['slope'] + rt_fxn_model['intercept']
            setsize = fxn.num_item

            for j in range(rt.shape[0]):
                data.append([rt[j], setsize[j], i])

        df = pd.DataFrame(data, columns = ['RT', 'SetSize', 'Task'])

        min_lm1 = ols('RT ~ SetSize', data=df).fit() #same slope same intercept
        min_lm2 = ols('RT ~ SetSize * Task', data = df).fit() #diff slope diff intercept
        table = anova_lm(min_lm1, min_lm2)
        
        modelTestData.append(table['Pr(>F)'][1])
        header.append(exp_name.replace(",", " - "))
        
    if n==0:
        testData.append(header)
        
    testData.append(modelTestData)

/home/shashi/anaconda3/envs/klab_vs/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/shashi/anaconda3/envs/klab_vs/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/shashi/anaconda3/envs/klab_vs/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [5]:
np.savetxt("results/significance_test.csv", testData, delimiter=',', fmt="%s")